In [167]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import cv2
from skimage.transform import resize
from skimage.io import imread
from sklearn.utils import shuffle
import random
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

### Loading Images

In [126]:
Categories=['rock','paper','scissors']
train_dir = 'rps-split\\train'
test_dir = 'rps-split\\test'
val_dir = 'rps-split\\val'

flat_data_arr=[] #input array
target_arr=[] #output array
for i in Categories:
    path=os.path.join(train_dir,i)
    for img in os.listdir(path):
        img_array=imread(os.path.join(path,img))
        img_resized=resize(img_array,(150,150,1))
        flat_data_arr.append(img_resized.flatten())
        target_arr.append(Categories.index(i))
    print(f'loaded category:{i} successfully for train')
flat_data=np.array(flat_data_arr)
target=np.array(target_arr)
df=pd.DataFrame(flat_data) #dataframe
df['Target']=target
x_train=df.iloc[:,:-1] #input data 
y_train=df.iloc[:,-1] #output data
x_train=x_train.sample(frac=1, random_state=42).reset_index(drop=True)
y_train=y_train.sample(frac=1, random_state=42).reset_index(drop=True)



flat_data_arr=[] #input array
target_arr=[] #output array
for i in Categories:
    path=os.path.join(test_dir,i)
    for img in os.listdir(path):
        img_array=imread(os.path.join(path,img))
        img_resized=resize(img_array,(150,150,1))
        flat_data_arr.append(img_resized.flatten())
        target_arr.append(Categories.index(i))
    print(f'loaded category:{i} successfully for test')
flat_data=np.array(flat_data_arr)
target=np.array(target_arr)
df=pd.DataFrame(flat_data) #dataframe
df['Target']=target
x_test=df.iloc[:,:-1] #input data 
y_test=df.iloc[:,-1] #output data

flat_data_arr=[] #input array
target_arr=[] #output array
for i in Categories:
    path=os.path.join(val_dir,i)
    for img in os.listdir(path):
        img_array=imread(os.path.join(path,img))
        img_resized=resize(img_array,(150,150,1))
        flat_data_arr.append(img_resized.flatten())
        target_arr.append(Categories.index(i))
    print(f'loaded category:{i} successfully for val')
flat_data=np.array(flat_data_arr)
target=np.array(target_arr)
df=pd.DataFrame(flat_data) #dataframe
df['Target']=target
x_val=df.iloc[:,:-1] #input data 
y_val=df.iloc[:,-1] #output data

loaded category:rock successfully for train
loaded category:paper successfully for train
loaded category:scissors successfully for train
loaded category:rock successfully for test
loaded category:paper successfully for test
loaded category:scissors successfully for test
loaded category:rock successfully for val
loaded category:paper successfully for val
loaded category:scissors successfully for val


In [131]:
x_train.shape

(1531, 22500)

In [129]:
x_train

,0,1,2,3,4,5,6,7,8,9,...,22490,22491,22492,22493,22494,22495,22496,22497,22498,22499
0,0.282439,0.289924,0.293366,0.300145,0.303655,0.308242,0.314224,0.317845,0.324846,0.325541,...,0.309310,0.307258,0.305002,0.299647,0.298464,0.293929,0.289883,0.283850,0.283182,0.269569
1,0.307829,0.299331,0.296886,0.301255,0.296857,0.306606,0.309610,0.308545,0.304403,0.308739,...,0.333674,0.332998,0.328856,0.331649,0.324068,0.326655,0.321872,0.321227,0.318055,0.321791
2,0.259973,0.268775,0.275280,0.280152,0.285039,0.285782,0.288814,0.292053,0.299367,0.303309,...,0.289264,0.288739,0.285866,0.280724,0.279225,0.273654,0.267650,0.263894,0.262009,0.252174
3,0.306726,0.303777,0.305048,0.305458,0.303881,0.309817,0.311885,0.309504,0.310891,0.314309,...,0.334475,0.337161,0.331765,0.335954,0.328828,0.325748,0.328762,0.326534,0.322945,0.324864
4,0.216188,0.223491,0.231423,0.240872,0.241220,0.249937,0.252278,0.255959,0.261174,0.265196,...,0.228960,0.225911,0.219907,0.219417,0.212889,0.205368,0.205186,0.201541,0.197776,0.189999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526,0.316758,0.319122,0.315090,0.317579,0.319337,0.326543,0.325820,0.322612,0.325771,0.332192,...,0.344872,0.343667,0.348252,0.339113,0.332841,0.333563,0.330319,0.335869,0.335691,0.332603
1527,0.382267,0.381571,0.378451,0.383468,0.384683,0.388388,0.387331,0.394883,0.393906,0.397792,...,0.425765,0.420005,0.422743,0.420270,0.416993,0.414020,0.414914,0.414313,0.415321,0.415032
1528,0.293139,0.287507,0.286387,0.289640,0.288580,0.296019,0.298054,0.285896,0.290841,0.299751,...,0.316646,0.314835,0.314268,0.311509,0.305610,0.309481,0.307940,0.304920,0.302529,0.301563
1529,0.304618,0.297201,0.304142,0.305809,0.306728,0.308514,0.317587,0.312711,0.313207,0.312118,...,0.332991,0.327786,0.325019,0.329533,0.324487,0.320900,0.321537,0.317153,0.316885,0.318734


In [130]:
y_train

0       1
1       0
2       0
3       2
4       1
       ..
1526    2
1527    2
1528    1
1529    2
1530    2
Name: Target, Length: 1531, dtype: int32

### Searching for best-performing model on training set

In [146]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid={'C':[0.1,1,10,100],'gamma':[0.0001,0.001,0.1,1],'kernel':['rbf','poly']}
#param_grid={'C':[0.1],'gamma':[0.0001],'kernel':['rbf','poly']}
model=GridSearchCV(SVC(),param_grid,refit=True,verbose=3)
model.fit(x_train, y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV 1/5] END ...C=0.1, gamma=0.0001, kernel=rbf;, score=0.342 total time=  32.1s
[CV 2/5] END ...C=0.1, gamma=0.0001, kernel=rbf;, score=0.343 total time=  30.8s
[CV 3/5] END ...C=0.1, gamma=0.0001, kernel=rbf;, score=0.343 total time=  30.4s
[CV 4/5] END ...C=0.1, gamma=0.0001, kernel=rbf;, score=0.343 total time=  41.5s
[CV 5/5] END ...C=0.1, gamma=0.0001, kernel=rbf;, score=0.343 total time=  33.5s
[CV 1/5] END ..C=0.1, gamma=0.0001, kernel=poly;, score=0.342 total time=  26.5s
[CV 2/5] END ..C=0.1, gamma=0.0001, kernel=poly;, score=0.343 total time=  24.0s
[CV 3/5] END ..C=0.1, gamma=0.0001, kernel=poly;, score=0.343 total time=  20.9s
[CV 4/5] END ..C=0.1, gamma=0.0001, kernel=poly;, score=0.343 total time=  23.4s
[CV 5/5] END ..C=0.1, gamma=0.0001, kernel=poly;, score=0.343 total time=  21.1s
[CV 1/5] END ....C=0.1, gamma=0.001, kernel=rbf;, score=0.704 total time=  32.7s
[CV 2/5] END ....C=0.1, gamma=0.001, kernel=rbf

[CV 2/5] END .......C=10, gamma=0.1, kernel=rbf;, score=0.392 total time=  32.3s
[CV 3/5] END .......C=10, gamma=0.1, kernel=rbf;, score=0.395 total time=  36.6s
[CV 4/5] END .......C=10, gamma=0.1, kernel=rbf;, score=0.392 total time=  31.5s
[CV 5/5] END .......C=10, gamma=0.1, kernel=rbf;, score=0.363 total time=  30.5s
[CV 1/5] END ......C=10, gamma=0.1, kernel=poly;, score=0.844 total time=  10.2s
[CV 2/5] END ......C=10, gamma=0.1, kernel=poly;, score=0.824 total time=  15.9s
[CV 3/5] END ......C=10, gamma=0.1, kernel=poly;, score=0.827 total time=  14.5s
[CV 4/5] END ......C=10, gamma=0.1, kernel=poly;, score=0.837 total time=  16.3s
[CV 5/5] END ......C=10, gamma=0.1, kernel=poly;, score=0.850 total time=   9.4s
[CV 1/5] END .........C=10, gamma=1, kernel=rbf;, score=0.342 total time=  28.8s
[CV 2/5] END .........C=10, gamma=1, kernel=rbf;, score=0.343 total time=  27.1s
[CV 3/5] END .........C=10, gamma=1, kernel=rbf;, score=0.343 total time=  32.2s
[CV 4/5] END .........C=10, 

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100],
                         'gamma': [0.0001, 0.001, 0.1, 1],
                         'kernel': ['rbf', 'poly']},
             verbose=3)

In [147]:
print(model.best_params_)

{'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}


In [139]:
from sklearn import svm
rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1).fit(x_train, y_train)
poly = svm.SVC(kernel='poly', degree=3, C=1).fit(x_train, y_train)

In [148]:
#model.fit(x_train,y_train)
y_pred=model.predict(x_test)
print("The predicted Data is :")
print(y_pred)
print("The actual data is:")
print(np.array(y_test))
#print(f"The model is {accuracy_score(y_pred,y_test)*100}% accurate")

The predicted Data is :
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 1 0 0
 0 0 0 2 0 0 0 0 0 0 0 2 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 1 0 2 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 0 1 1 1 1 0 1 1 1 2 1 0 1 1 1 2 1 1 1
 1 1 1 1 2 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 1 1 1 2 2 2 2 2 0
 2 2 2 2 2 2 2 1 2 0 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2
 2 2 0 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
The actual data is:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

In [149]:
def score(trained_model):
    pred_y = trained_model.predict(x_test)
    print(confusion_matrix(y_test,pred_y))
    print(classification_report(y_test,pred_y))


In [150]:
#C=100, gamma=0.001, kernel=rbf
score(model)

[[103   1   5]
 [  7  93   7]
 [  7   1 105]]
              precision    recall  f1-score   support

           0       0.88      0.94      0.91       109
           1       0.98      0.87      0.92       107
           2       0.90      0.93      0.91       113

    accuracy                           0.91       329
   macro avg       0.92      0.91      0.92       329
weighted avg       0.92      0.91      0.92       329



### Testing alternative models

In [159]:
#C=0.1, gamma=0.001, kernel=poly
score(svm.SVC(C=0.1, kernel='poly').fit(x_train, y_train))

[[101   1   7]
 [  7  92   8]
 [ 10   8  95]]
              precision    recall  f1-score   support

           0       0.86      0.93      0.89       109
           1       0.91      0.86      0.88       107
           2       0.86      0.84      0.85       113

    accuracy                           0.88       329
   macro avg       0.88      0.88      0.88       329
weighted avg       0.88      0.88      0.88       329



In [158]:
score(svm.SVC(C=1, kernel='poly').fit(x_train, y_train))

[[102   1   6]
 [  9  90   8]
 [ 10   8  95]]
              precision    recall  f1-score   support

           0       0.84      0.94      0.89       109
           1       0.91      0.84      0.87       107
           2       0.87      0.84      0.86       113

    accuracy                           0.87       329
   macro avg       0.87      0.87      0.87       329
weighted avg       0.87      0.87      0.87       329



In [160]:
score(svm.SVC(C=10, kernel='poly').fit(x_train, y_train))

[[102   1   6]
 [  9  90   8]
 [ 10   8  95]]
              precision    recall  f1-score   support

           0       0.84      0.94      0.89       109
           1       0.91      0.84      0.87       107
           2       0.87      0.84      0.86       113

    accuracy                           0.87       329
   macro avg       0.87      0.87      0.87       329
weighted avg       0.87      0.87      0.87       329



In [161]:
score(svm.SVC(C=100, kernel='poly').fit(x_train, y_train))

[[102   1   6]
 [  9  90   8]
 [ 10   8  95]]
              precision    recall  f1-score   support

           0       0.84      0.94      0.89       109
           1       0.91      0.84      0.87       107
           2       0.87      0.84      0.86       113

    accuracy                           0.87       329
   macro avg       0.87      0.87      0.87       329
weighted avg       0.87      0.87      0.87       329



In [162]:
score(svm.SVC(C=0.1, gamma=0.001, kernel='rbf').fit(x_train, y_train))

[[72 13 24]
 [13 72 22]
 [17  2 94]]
              precision    recall  f1-score   support

           0       0.71      0.66      0.68       109
           1       0.83      0.67      0.74       107
           2       0.67      0.83      0.74       113

    accuracy                           0.72       329
   macro avg       0.73      0.72      0.72       329
weighted avg       0.73      0.72      0.72       329



In [163]:
score(svm.SVC(C=1, gamma=0.001, kernel='rbf').fit(x_train, y_train))

[[ 95   3  11]
 [  5  88  14]
 [  9   1 103]]
              precision    recall  f1-score   support

           0       0.87      0.87      0.87       109
           1       0.96      0.82      0.88       107
           2       0.80      0.91      0.85       113

    accuracy                           0.87       329
   macro avg       0.88      0.87      0.87       329
weighted avg       0.88      0.87      0.87       329



In [164]:
score(svm.SVC(C=10, gamma=0.001, kernel='rbf').fit(x_train, y_train))

[[103   1   5]
 [  5  94   8]
 [  7   3 103]]
              precision    recall  f1-score   support

           0       0.90      0.94      0.92       109
           1       0.96      0.88      0.92       107
           2       0.89      0.91      0.90       113

    accuracy                           0.91       329
   macro avg       0.91      0.91      0.91       329
weighted avg       0.91      0.91      0.91       329



In [165]:
score(svm.SVC(C=0.1, gamma=0.0001, kernel='rbf').fit(x_train, y_train))

[[  0   0 109]
 [  0   0 107]
 [  0   0 113]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       109
           1       0.00      0.00      0.00       107
           2       0.34      1.00      0.51       113

    accuracy                           0.34       329
   macro avg       0.11      0.33      0.17       329
weighted avg       0.12      0.34      0.18       329



C:\Users\jcrul\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jcrul\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jcrul\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### RBF with c=100, gamma=0.001 still the most accurate with 91-92% on testing data
### c=0.001 classifies all into scissors

### Validation accuracy

In [170]:
y_pred_val = model.predict(x_val)
print(confusion_matrix(y_val,y_pred_val))
print(classification_report(y_val,y_pred_val))

[[ 98   3   8]
 [  4  92  11]
 [  3   1 108]]
              precision    recall  f1-score   support

           0       0.93      0.90      0.92       109
           1       0.96      0.86      0.91       107
           2       0.85      0.96      0.90       112

    accuracy                           0.91       328
   macro avg       0.91      0.91      0.91       328
weighted avg       0.91      0.91      0.91       328

